<a href="https://colab.research.google.com/github/matheusmaloja/Tese-POF/blob/main/Tese2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. PUXANDO AS BIBLIOTECAS

In [ ]:
# In[ ]: Importação dos pacotes necessários

import pandas as pd # manipulação de dado em formato de dataframe
import seaborn as sns # biblioteca de visualização de informações estatísticas
import matplotlib.pyplot as plt # biblioteca de visualização de dados
import statsmodels.api as sm # biblioteca de modelagem estatística
import numpy as np # biblioteca para operações matemáticas multidimensionais
from scipy import stats # estatística chi2
from statsmodels.iolib.summary2 import summary_col # comparação entre modelos
import plotly.graph_objs as go # gráfico 3D
import statsmodels.formula.api as smf # estimação do modelo logístico binário

import warnings
warnings.filterwarnings('ignore')

2. TRAZENDO OS BANCOS DE DADOS

In [ ]:
df = pd.read_csv('df.csv', delimiter = ',') #tabela principal
df_nivel1 = pd.read_csv('gasto_nivel1.csv', delimiter = ',') #Gastos a nível 1
df_nivel2 = pd.read_csv('gasto_nivel2.csv', delimiter = ',') #Gastos a nível 2
df_nivel3 = pd.read_csv('gasto_nivel3.csv', delimiter = ',') #Gastos a nível 3


3. CRIANDO VARIÁVEIS BINÁRIAS E POR QUINTIS
*   escassez_b = '0' Escassez e '1' Não escassez
*   quintil_renda = quintos de renda disponível per capita
*   escassez_q = quintos de escassez (renda bruta - V0601)


In [ ]:
#Binarizando a variável escassez
df['escassez_b'] = np.where(df['escassez'] < 0, 0, 1)

In [ ]:
#Quintos de renda
df['quintil_renda'] = pd.qcut(df['renda_disponivel_per_capita'], q=5, labels=[1, 2, 3, 4, 5]).reset_index(drop=True)
df.quintil_renda

,quintil_renda
0,1
1,1
2,2
3,1
4,2
...,...
58024,2
58025,2
58026,1
58027,2


In [ ]:
#Criando Faixas
df['escassez_q'] = pd.qcut(df['escassez'], q=5, labels=['1', '2', '3', '4', '5'])
df['escassez_q'].value_counts().sort_index()

,count
escassez_q,
1,11606
2,11606
3,11605
4,11606
5,11606


4. FAZENDO MERGES

In [ ]:
df_m1 = df.merge(df_nivel1, left_on='id_domicilio', right_on='dom_id', suffixes=('_df', '_nivel1'))
df_m1

,sigla_uf,situacao,id_estrato,id_unidade_primaria_amostragem,id_domicilio,id_unidade_consumo,id_informante,V61041,V61042,V61043,...,renda_nao_monetaria_per_capita,renda_monetaria_per_capita,renda_disponivel_per_capita,escassez,escassez_b,quintil_renda,escassez_q,dom_id,Descricao_1,gasto
0,PA,2,1514,150000410,15000041001,1500004100101,150000410010102,2,1,3,...,83.918958,90.167500,112.505417,-466.35,0,1,2,15000041001,Alimentação fora do domicílio,142.263333
1,PA,2,1514,150000410,15000041001,1500004100101,150000410010102,2,1,3,...,83.918958,90.167500,112.505417,-466.35,0,1,2,15000041001,Alimentação no domicílio,487.716667
2,PA,2,1514,150000410,15000041002,1500004100201,150000410020102,2,1,2,...,246.140833,135.630000,381.226667,-236.46,0,1,2,15000041002,Alimentação fora do domicílio,4.463333
3,PA,2,1514,150000410,15000041002,1500004100201,150000410020102,2,1,2,...,246.140833,135.630000,381.226667,-236.46,0,1,2,15000041002,Alimentação no domicílio,286.780000
4,PA,2,1514,150000410,15000041003,1500004100301,150000410030101,3,3,3,...,137.393333,541.136250,678.529583,63.69,1,2,3,15000041003,Alimentação no domicílio,168.783333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85209,PE,1,2601,260129517,26012951708,2601295170801,260129517080102,2,2,1,...,357.502167,57.761333,397.603833,-675.76,0,1,2,26012951708,Alimentação fora do domicílio,347.230000
85210,PE,1,2601,260129517,26012951708,2601295170801,260129517080102,2,2,1,...,357.502167,57.761333,397.603833,-675.76,0,1,2,26012951708,Alimentação no domicílio,181.393333
85211,PE,1,2601,260129517,26012951709,2601295170901,260129517090102,2,2,2,...,116.391944,559.308611,623.763056,527.10,1,2,3,26012951709,Alimentação no domicílio,63.050000
85212,PE,1,2601,260129517,26012951713,2601295171301,260129517130101,2,2,2,...,46.145000,224.244583,265.558333,-869.75,0,1,1,26012951713,Alimentação fora do domicílio,12.826667


In [ ]:
df_m2 = df.merge(df_nivel2, left_on='id_domicilio', right_on='dom_id', suffixes=('_df', '_nivel2'))
df_m2

,sigla_uf,situacao,id_estrato,id_unidade_primaria_amostragem,id_domicilio,id_unidade_consumo,id_informante,V61041,V61042,V61043,...,renda_monetaria_per_capita,renda_disponivel_per_capita,escassez,escassez_b,quintil_renda,escassez_q,dom_id,Nivel_2,Descricao_2,gasto
0,PA,2,1514,150000410,15000041001,1500004100101,150000410010102,2,1,3,...,90.167500,112.505417,-466.35,0,1,2,15000041001,101,"Cereais, leguminosas e oleaginosas",12.393333
1,PA,2,1514,150000410,15000041001,1500004100101,150000410010102,2,1,3,...,90.167500,112.505417,-466.35,0,1,2,15000041001,102,"Farinhas, féculas e massas",72.713333
2,PA,2,1514,150000410,15000041001,1500004100101,150000410010102,2,1,3,...,90.167500,112.505417,-466.35,0,1,2,15000041001,104,Açúcares e derivados,20.366667
3,PA,2,1514,150000410,15000041001,1500004100101,150000410010102,2,1,3,...,90.167500,112.505417,-466.35,0,1,2,15000041001,106,Frutas,28.903333
4,PA,2,1514,150000410,15000041001,1500004100101,150000410010102,2,1,3,...,90.167500,112.505417,-466.35,0,1,2,15000041001,108,Aves e ovos,106.253333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382713,PE,1,2601,260129517,26012951713,2601295171301,260129517130101,2,2,2,...,224.244583,265.558333,-869.75,0,1,1,26012951713,111,Óleos e gorduras,18.763333
382714,PE,1,2601,260129517,26012951713,2601295171301,260129517130101,2,2,2,...,224.244583,265.558333,-869.75,0,1,1,26012951713,112,Bebidas e infusões,56.246667
382715,PE,1,2601,260129517,26012951713,2601295171301,260129517130101,2,2,2,...,224.244583,265.558333,-869.75,0,1,1,26012951713,114,Sal e condimentos,53.820000
382716,PE,1,2601,260129517,26012951713,2601295171301,260129517130101,2,2,2,...,224.244583,265.558333,-869.75,0,1,1,26012951713,203,Sanduíches e salgados,8.623333


In [ ]:
df_m3 = df.merge(df_nivel3, left_on='id_domicilio', right_on='dom_id', suffixes=('_df', '_nivel3'))
df_m3

,sigla_uf,situacao,id_estrato,id_unidade_primaria_amostragem,id_domicilio,id_unidade_consumo,id_informante,V61041,V61042,V61043,...,escassez,escassez_b,quintil_renda,escassez_q,dom_id,Descricao_2,Descricao_3,Nivel_2,Nivel_3,gasto
0,PA,2,1514,150000410,15000041001,1500004100101,150000410010102,2,1,3,...,-466.35,0,1,2,15000041001,Almoço e jantar,NaN,201,NaN,61.966667
1,PA,2,1514,150000410,15000041001,1500004100101,150000410010102,2,1,3,...,-466.35,0,1,2,15000041001,Aves e ovos,Ovo de galinha,108,1082.0,106.253333
2,PA,2,1514,150000410,15000041001,1500004100101,150000410010102,2,1,3,...,-466.35,0,1,2,15000041001,Açúcares e derivados,Açúcar cristal,104,1042.0,20.366667
3,PA,2,1514,150000410,15000041001,1500004100101,150000410010102,2,1,3,...,-466.35,0,1,2,15000041001,Bebidas e infusões,Café moído,112,1121.0,23.270000
4,PA,2,1514,150000410,15000041001,1500004100101,150000410010102,2,1,3,...,-466.35,0,1,2,15000041001,Bebidas e infusões,Refrigerantes,112,1122.0,62.356667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543159,PE,1,2601,260129517,26012951713,2601295171301,260129517130101,2,2,2,...,-869.75,0,1,1,26012951713,Sal e condimentos,Sal refinado,114,1143.0,4.376667
543160,PE,1,2601,260129517,26012951713,2601295171301,260129517130101,2,2,2,...,-869.75,0,1,1,26012951713,Sanduíches e salgados,NaN,203,NaN,8.623333
543161,PE,1,2601,260129517,26012951713,2601295171301,260129517130101,2,2,2,...,-869.75,0,1,1,26012951713,Tubérculos e raízes,Batata inglesa,103,1031.0,17.723333
543162,PE,1,2601,260129517,26012951713,2601295171301,260129517130101,2,2,2,...,-869.75,0,1,1,26012951713,Tubérculos e raízes,Cenoura,103,1032.0,1.300000


5. CRIANDO REGIÕES

In [ ]:
# Criando coluna para região
regiao = {
    # Norte
    "AC": "Norte", "AP": "Norte", "AM": "Norte", "PA": "Norte",
    "RO": "Norte", "RR": "Norte", "TO": "Norte",

    # Nordeste
    "AL": "Nordeste", "BA": "Nordeste", "CE": "Nordeste", "MA": "Nordeste",
    "PB": "Nordeste", "PE": "Nordeste", "PI": "Nordeste",
    "RN": "Nordeste", "SE": "Nordeste",

    # Centro-Oeste
    "DF": "Centro-Oeste", "GO": "Centro-Oeste",
    "MT": "Centro-Oeste", "MS": "Centro-Oeste",

    # Sudeste
    "ES": "Sudeste", "MG": "Sudeste", "RJ": "Sudeste", "SP": "Sudeste",

    # Sul
    "PR": "Sul", "RS": "Sul", "SC": "Sul"
}

# Supondo que sua coluna de siglas seja chamada 'UF':
df['regiao'] = df['sigla_uf'].map(regiao)

6. ANALISANDO VARIÁVEIS DESCRITIVAS - DADOS BINÁRIOS

In [ ]:
#Quintil de renda por grupos
renda_quintil = df.groupby(['escassez_b', 'quintil_renda']).size().reset_index(name='n')
renda_quintil

,escassez_b,quintil_renda,n
0,0,1,8228
1,0,2,5734
2,0,3,4501
3,0,4,3483
4,0,5,2164
5,1,1,3378
6,1,2,5872
7,1,3,7104
8,1,4,8123
9,1,5,9442


In [ ]:
#Médias com IC 95%
dados = df[df['escassez_b'] == 1]['anos_estudo'].dropna() #Trocas no ['anos_estudo'] por 'renda_bruta', 'idade', 'renda disponivel percapita', 'escassez', 'anos estudo'
media = np.mean(dados)
sem = np.std(dados, ddof=1) / np.sqrt(len(dados))  # erro padrão
z = stats.norm.ppf(0.975)  # z para 95%
ic_inferior = media - z * sem
ic_superior = media + z * sem
print(f"Média: {media:.2f}")
print(f"IC 95%: ({ic_inferior:.2f}, {ic_superior:.2f})")

Média: 8.91
IC 95%: (8.85, 8.96)


In [ ]:
#Sexo
sexo_count = df.groupby('escassez_b')['sexo'].value_counts().reset_index()
sexo_count

,escassez_b,sexo,count
0,0,2,14519
1,0,1,9591
2,1,2,20076
3,1,1,13843


In [ ]:
#Raça
raca_count = df.groupby('escassez_b')['raca_cor'].value_counts().reset_index()
raca_count

,escassez_b,raca_cor,count
0,0,4,13069
1,0,1,7843
2,0,2,2907
3,0,5,133
4,0,3,128
5,0,9,30
6,1,4,16012
7,1,1,13966
8,1,2,3527
9,1,5,181


In [ ]:
#Instrulçao
instrucao = df.groupby('escassez_b')['nivel_instrucao'].value_counts().reset_index()
instrucao

,escassez_b,nivel_instrucao,count
0,0,2,9948
1,0,5,5550
2,0,1,2477
3,0,3,2065
4,0,7,2038
5,0,4,1320
6,0,6,712
7,1,2,13208
8,1,5,7619
9,1,7,4763


In [ ]:
#Escassez por quintil
contagem = df.groupby(['escassez_b', 'quintil_renda']).size().reset_index(name='count')
print(contagem)


   escassez_b quintil_renda  count
0           0             1   8228
1           0             2   5734
2           0             3   4501
3           0             4   3483
4           0             5   2164
5           1             1   3378
6           1             2   5872
7           1             3   7104
8           1             4   8123
9           1             5   9442


7. Analisando Quintis de Renda


In [ ]:
renda_r = df.groupby(['quintil_renda'])[['escassez', 'renda_disponivel_per_capita', 'renda_total']].mean().reset_index()
renda_r

,quintil_renda,escassez,renda_disponivel_per_capita,renda_total
0,1,-599.393384,370.092978,1431.987022
1,2,-48.143739,757.752510,2538.044870
2,3,384.798236,1154.296731,3388.716030
3,4,1068.005409,1707.635769,4667.739685
4,5,4842.725048,4415.999064,11325.219534


In [ ]:
import numpy as np
from scipy import stats

# Exemplo com uma grande amostra
dados = df[df['quintil_renda'] == 5]['anos_estudo'].dropna() #Trocas no ['anos_estudo'] por 'renda_bruta', 'idade', 'renda disponivel percapita', 'escassez', 'anos estudo'

# Parâmetros
media = np.mean(dados)
sem = np.std(dados, ddof=1) / np.sqrt(len(dados))  # erro padrão
z = stats.norm.ppf(0.975)  # z para 95%

# Intervalo de confiança
ic_inferior = media - z * sem
ic_superior = media + z * sem

print(f"Média: {media:.2f}")
print(f"IC 95%: ({ic_inferior:.2f}, {ic_superior:.2f})")

Média: 11.51
IC 95%: (11.42, 11.60)


In [ ]:
#Sexo
sexo_count = df.groupby('quintil_renda')['sexo'].value_counts().reset_index()
sexo_count

,quintil_renda,sexo,count
0,1,2,7556
1,1,1,4050
2,2,2,7295
3,2,1,4311
4,3,2,6832
5,3,1,4773
6,4,2,6614
7,4,1,4992
8,5,2,6298
9,5,1,5308


In [ ]:
#Raça
raca_count = df.groupby('quintil_renda')['raca_cor'].value_counts().reset_index()
raca_count

,quintil_renda,raca_cor,count
0,1,4,7482
1,1,1,2485
2,1,2,1473
3,1,5,91
4,1,3,64
5,1,9,11
6,2,4,6633
7,2,1,3343
8,2,2,1511
9,2,5,59


In [ ]:
#Instrulçao
instrucao = df.groupby('quintil_renda')['nivel_instrucao'].value_counts().reset_index()
instrucao

,quintil_renda,nivel_instrucao,count
0,1,2,5846
1,1,5,2118
2,1,1,1452
3,1,3,1065
4,1,4,756
5,1,7,223
6,1,6,146
7,2,2,5241
8,2,5,2624
9,2,1,1299


8. Qualidade de Vida e Alimentação

In [ ]:
avaliacao_renda = df.groupby('quintil_renda')['V6101'].value_counts().reset_index()
avaliacao_renda

,quintil_renda,V6101,count
0,1,3,3901
1,1,2,3451
2,1,1,2871
3,1,4,807
4,1,5,528
5,1,6,48
6,2,3,4450
7,2,2,3079
8,2,1,1951
9,2,4,1319


In [ ]:
avaliacao_renda_binaria = df.groupby('escassez_b')['V6101'].value_counts().reset_index()
avaliacao_renda_binaria

,escassez_b,V6101,count
0,0,3,8999
1,0,2,6500
2,0,1,4558
3,0,4,2552
4,0,5,1394
5,0,6,107
6,1,3,11748
7,1,4,6544
8,1,2,6012
9,1,5,5550


In [ ]:
avaliacao_alimentacao = df.groupby('quintil_renda')['V61041'].value_counts().reset_index()
avaliacao_alimentacao

,quintil_renda,V61041,count
0,1,2,5198
1,1,1,5170
2,1,3,1238
3,2,1,6129
4,2,2,4698
5,2,3,779
6,3,1,6748
7,3,2,4281
8,3,3,576
9,4,1,7307


In [ ]:
avaliacao_alimentacao_binaria = df.groupby('escassez_b')['V61041'].value_counts().reset_index()
avaliacao_alimentacao_binaria

,escassez_b,V61041,count
0,0,1,12303
1,0,2,9887
2,0,3,1920
3,1,1,21615
4,1,2,10937
5,1,3,1367


9. Regressão logística

In [ ]:
x = df.escassez
y = df.V6101
#Esse pacote precisa que a constante seja definida pelo usuário
X = sm.add_constant(x)
#Estimação do modelo - função 'MNLogit' do pacote
#'statsmodels.discrete.discrete_model'
from statsmodels.discrete.discrete_model import MNLogit
modelo = MNLogit(endog=y, exog=X).fit()

#Parâmetros do modelo
modelo.summary()

Optimization terminated successfully.
         Current function value: 1.533495
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                  V6101   No. Observations:                58029
Model:                        MNLogit   Df Residuals:                    58019
Method:                           MLE   Df Model:                            5
Date:                Sat, 14 Jun 2025   Pseudo R-squ.:                 0.02291
Time:                        16:12:12   Log-Likelihood:                -88987.
converged:                       True   LL-Null:                       -91074.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
   V6101=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4405      0.014     30.804      0.000       0.412       0.469
escassez    1.702e-05   4.68e-06      3.633      0.000    7.84e-06    2.62e-05
------------------------------------------------------------------------------
   V6101=3       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9165      0.013     69.135      0.000       0.891       0.943
escassez    8.774e-05   4.79e-06     18.327      0.000    7.84e-05    9.71e-05
------------------------------------------------------------------------------
   V6101=4       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0322      0.016     -2.008      0.045      -0.064      -0.001
escassez       0.0002   5.06e-06     37.414      0.000       0.000       0.000
------------------------------------------------------------------------------
   V6101=5       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3870      0.018    -22.096      0.000      -0.421      -0.353
escassez       0.0002   5.09e-06     43.521      0.000       0.000       0.000
------------------------------------------------------------------------------
   V6101=6       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.7624      0.042    -65.514      0.000      -2.845      -2.680
escassez       0.0002   5.47e-06     42.651      0.000       0.000       0.000
==============================================================================
"""

In [ ]:
x = df.escassez
y = df.V61041
#Esse pacote precisa que a constante seja definida pelo usuário
X = sm.add_constant(x)
#Estimação do modelo - função 'MNLogit' do pacote
#'statsmodels.discrete.discrete_model'
from statsmodels.discrete.discrete_model import MNLogit
modelo = MNLogit(endog=y, exog=X).fit()

#Parâmetros do modelo
modelo.summary()

Optimization terminated successfully.
         Current function value: 0.836209
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                 V61041   No. Observations:                58029
Model:                        MNLogit   Df Residuals:                    58025
Method:                           MLE   Df Model:                            2
Date:                Sat, 14 Jun 2025   Pseudo R-squ.:                0.009543
Time:                        16:12:12   Log-Likelihood:                -48524.
converged:                       True   LL-Null:                       -48992.
Covariance Type:            nonrobust   LLR p-value:                8.930e-204
==============================================================================
  V61041=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4295      0.009    -47.278      0.000      -0.447      -0.412
escassez   -6.228e-05   2.69e-06    -23.159      0.000   -6.76e-05    -5.7e-05
------------------------------------------------------------------------------
  V61041=3       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.2638      0.018   -123.026      0.000      -2.300      -2.228
escassez      -0.0001   5.46e-06    -18.977      0.000      -0.000    -9.3e-05
==============================================================================
"""

In [ ]:
x = df.renda_disponivel_per_capita
y = df.V6101
#Esse pacote precisa que a constante seja definida pelo usuário
X = sm.add_constant(x)
#Estimação do modelo - função 'MNLogit' do pacote
#'statsmodels.discrete.discrete_model'
from statsmodels.discrete.discrete_model import MNLogit
modelo = MNLogit(endog=y, exog=X).fit()

#Parametros do modelo
modelo.summary()

Optimization terminated successfully.
         Current function value: 1.510739
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                  V6101   No. Observations:                58029
Model:                        MNLogit   Df Residuals:                    58019
Method:                           MLE   Df Model:                            5
Date:                Sat, 14 Jun 2025   Pseudo R-squ.:                 0.03741
Time:                        16:12:21   Log-Likelihood:                -87667.
converged:                       True   LL-Null:                       -91074.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===============================================================================================
                    V6101=2       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           0.1626      0.024      6.653      0.000       0.115       0.211
renda_disponivel_per_capita     0.0003   1.85e-05     13.765      0.000       0.000       0.000
-----------------------------------------------------------------------------------------------
                    V6101=3       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           0.3099      0.023     13.710      0.000       0.266       0.354
renda_disponivel_per_capita     0.0005   1.69e-05     31.252      0.000       0.000       0.001
-----------------------------------------------------------------------------------------------
                    V6101=4       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          -0.9356      0.026    -36.626      0.000      -0.986      -0.886
renda_disponivel_per_capita     0.0008   1.72e-05     44.566      0.000       0.001       0.001
-----------------------------------------------------------------------------------------------
                    V6101=5       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          -1.3869      0.027    -51.312      0.000      -1.440      -1.334
renda_disponivel_per_capita     0.0008   1.73e-05     48.649      0.000       0.001       0.001
-----------------------------------------------------------------------------------------------
                    V6101=6       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          -3.9582      0.052    -75.828      0.000      -4.061      -3.856
renda_disponivel_per_capita     0.0009   1.81e-05     50.106      0.000       0.001       0.001
===============================================================================================
"""

In [ ]:
x = df.renda_disponivel_per_capita
y = df.V61041
#Esse pacote precisa que a constante seja definida pelo usuário
X = sm.add_constant(x)
#Estimação do modelo - função 'MNLogit' do pacote
#'statsmodels.discrete.discrete_model'
from statsmodels.discrete.discrete_model import MNLogit
modelo = MNLogit(endog=y, exog=X).fit()

#Parametros do modelo
modelo.summary()

Optimization terminated successfully.
         Current function value: 0.825383
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                 V61041   No. Observations:                58029
Model:                        MNLogit   Df Residuals:                    58025
Method:                           MLE   Df Model:                            2
Date:                Sat, 14 Jun 2025   Pseudo R-squ.:                 0.02237
Time:                        16:12:22   Log-Likelihood:                -47896.
converged:                       True   LL-Null:                       -48992.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===============================================================================================
                   V61041=2       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          -0.1368      0.013    -10.220      0.000      -0.163      -0.111
renda_disponivel_per_capita    -0.0002   6.98e-06    -32.315      0.000      -0.000      -0.000
-----------------------------------------------------------------------------------------------
                   V61041=3       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          -1.5614      0.031    -50.219      0.000      -1.622      -1.500
renda_disponivel_per_capita    -0.0006   2.39e-05    -24.695      0.000      -0.001      -0.001
===============================================================================================
"""

10. Análise da alimentação

In [ ]:
# Função para calcular média e intervalo de confiança
def media_ic(grupo):
    media = grupo.mean()
    n = grupo.count()
    sem = grupo.std(ddof=1) / np.sqrt(n)  # erro padrão
    z = stats.norm.ppf(0.975)  # z para 95%
    margem_erro = z * sem
    return pd.Series({
        'media': media,
        'ic_inferior': media - margem_erro,
        'ic_superior': media + margem_erro
    })

# Aplicando a função por grupo
media_ic_gasto = df_m1.groupby(['escassez_b', 'Descricao_1'])['gasto'].apply(media_ic).reset_index()

print(media_ic_gasto)

    escassez_b                    Descricao_1      level_2       gasto
0            0  Alimentação fora do domicílio        media  214.568447
1            0  Alimentação fora do domicílio  ic_inferior  209.287339
2            0  Alimentação fora do domicílio  ic_superior  219.849554
3            0       Alimentação no domicílio        media  412.850905
4            0       Alimentação no domicílio  ic_inferior  406.754057
5            0       Alimentação no domicílio  ic_superior  418.947753
6            1  Alimentação fora do domicílio        media  363.412108
7            1  Alimentação fora do domicílio  ic_inferior  356.885195
8            1  Alimentação fora do domicílio  ic_superior  369.939022
9            1       Alimentação no domicílio        media  537.818923
10           1       Alimentação no domicílio  ic_inferior  531.509128
11           1       Alimentação no domicílio  ic_superior  544.128719


In [ ]:
# Função para calcular média e intervalo de confiança
def media_ic(grupo):
    media = grupo.mean()
    n = grupo.count()
    sem = grupo.std(ddof=1) / np.sqrt(n)  # erro padrão
    z = stats.norm.ppf(0.975)  # z para 95%
    margem_erro = z * sem
    return pd.Series({
        'media': media,
        'ic_inferior': media - margem_erro,
        'ic_superior': media + margem_erro
    })

# Aplicando a função por grupo
media_ic_gasto = df_m1.groupby(['quintil_renda', 'Descricao_1'])['gasto'].apply(media_ic).reset_index()

print(media_ic_gasto)

   quintil_renda                    Descricao_1      level_2       gasto
0              1  Alimentação fora do domicílio        media  147.276289
1              1  Alimentação fora do domicílio  ic_inferior  142.346129
2              1  Alimentação fora do domicílio  ic_superior  152.206449
3              1       Alimentação no domicílio        media  383.751501
4              1       Alimentação no domicílio  ic_inferior  375.947316
5              1       Alimentação no domicílio  ic_superior  391.555686
6              2  Alimentação fora do domicílio        media  214.582044
7              2  Alimentação fora do domicílio  ic_inferior  208.100328
8              2  Alimentação fora do domicílio  ic_superior  221.063761
9              2       Alimentação no domicílio        media  424.633971
10             2       Alimentação no domicílio  ic_inferior  415.992922
11             2       Alimentação no domicílio  ic_superior  433.275020
12             3  Alimentação fora do domicílio    

In [ ]:
media_gasto = df_m2.groupby(['escassez_b', 'Descricao_2'])['gasto'].mean().reset_index()
media_gasto = media_gasto.merge(
    df_m2[['Descricao_2', 'Nivel_2']].drop_duplicates(),
    on='Descricao_2',
    how='left'
)
media_gasto

,escassez_b,Descricao_2,gasto,Nivel_2
0,0,Alimentação light e diet,24.479796,207
1,0,Alimentos preparados,86.326156,115
2,0,Almoço e jantar,199.078049,201
3,0,Aves e ovos,85.439007,108
4,0,Açúcares e derivados,43.670287,104
5,0,Bebidas e infusões,73.474509,112
6,0,"Café, leite, café/leite e chocolate",19.862878,202
7,0,"Carnes, vísceras e pescados",160.745721,107
8,0,"Cereais, leguminosas e oleaginosas",65.631430,101
9,0,"Cervejas, chopes e outras bebidas alcoólicas",141.815264,206


In [ ]:
media_gasto_q = df_m2.groupby(['quintil_renda', 'Descricao_2'])['gasto'].mean().reset_index()
media_gasto_q = media_gasto_q.merge(
    df_m2[['Descricao_2', 'Nivel_2']].drop_duplicates(),
    on='Descricao_2',
    how='left'
)
media_gasto_q

,quintil_renda,Descricao_2,gasto,Nivel_2
0,1,Alimentação light e diet,20.537833,207
1,1,Alimentos preparados,67.259950,115
2,1,Almoço e jantar,127.382856,201
3,1,Aves e ovos,90.181161,108
4,1,Açúcares e derivados,35.784933,104
...,...,...,...,...
115,5,Refrigerantes e outras bebidas não alcoólicas,50.063446,204
116,5,Sal e condimentos,40.377255,114
117,5,Sanduíches e salgados,109.000544,203
118,5,Tubérculos e raízes,32.031309,103


11. Ultraprocessados

In [ ]:
media_gasto = df_m3.groupby(['escassez_b', 'Nivel_2', 'Nivel_3', 'Descricao_2',
                             'Descricao_3'])['gasto'].mean().reset_index()
media_gasto



,escassez_b,Nivel_2,Nivel_3,Descricao_2,Descricao_3,gasto
0,0,101,1011.0,"Cereais, leguminosas e oleaginosas",Arroz,54.311592
1,0,101,1012.0,"Cereais, leguminosas e oleaginosas",Feijao,26.093953
2,0,101,1012.0,"Cereais, leguminosas e oleaginosas",Feijão,34.241118
3,0,101,1013.0,"Cereais, leguminosas e oleaginosas",Orgânicos,33.510533
4,0,101,1014.0,"Cereais, leguminosas e oleaginosas",Outros,27.055118
...,...,...,...,...,...,...
111,1,112,1126.0,Bebidas e infusões,Outras,55.240697
112,1,114,1141.0,Sal e condimentos,Massa de tomate,20.108204
113,1,114,1142.0,Sal e condimentos,Maionese,23.774477
114,1,114,1143.0,Sal e condimentos,Sal refinado,7.971399


In [ ]:
def classificar_nova(row):
    desc_3 = str(row['Descricao_3']).lower()
    desc_2 = str(row['Descricao_2']).lower()

    # Grupo 1 – In natura ou minimamente processados
    if any(x in desc_3 for x in ['arroz', 'feijão', 'feijao', 'orgânico', 'batata', 'cenoura', 'mandioca',
                                 'cebola', 'tomate', 'alface', 'banana', 'laranja', 'maçã', 'maca',
                                 'frango', 'ovo', 'leite de vaca', 'pescado', 'carne de boi', 'carne de suíno']):
        return 1
    elif 'outras frutas' in desc_3:
        return 1
    elif 'café moído' in desc_3:
        return 1
    elif 'outros' in desc_3 and desc_2 in ['cereais, leguminosas e oleaginosas',
                                           'tubérculos e raízes',
                                           'legumes e verduras',
                                           'frutas',
                                           'carnes, vísceras e pescados',
                                           'aves e ovos']:
        return 1

    # Grupo 2 – Ingredientes culinários processados
    elif any(x in desc_3 for x in ['farinha', 'açúcar', 'acucar', 'óleo', 'oleo', 'azeite', 'sal']):
        return 2
    elif 'outras' in desc_3 and desc_2 == 'farinhas, féculas e massas':
        return 2
    elif 'outros' in desc_3 and desc_2 in ['açúcares e derivados', 'óleos e gorduras', 'sal e condimentos']:
        return 2

    # Grupo 3 – Processados
    elif any(x in desc_3 for x in ['pão', 'queijo']):
        return 3
    elif 'leite em pó' in desc_3:
        return 3
    elif 'outros' in desc_3 and desc_2 in ['panificados', 'leites e derivados']:
        return 3

    # Grupo 4 – Ultraprocessados
    elif any(x in desc_3 for x in ['refrigerante', 'biscoito', 'macarrão', 'massa de tomate', 'maionese',
                                   'light', 'diet', 'cerveja', 'bebida', 'industrializados']):
        return 4
    elif 'outras' in desc_3 and desc_2 in ['bebidas e infusões', 'panificados', 'sal e condimentos']:
        return 4
    elif desc_3 == 'outras' and desc_2 == 'bebidas e infusões':
        return 4

    else:
        return None  # caso não consiga classificar


In [ ]:
# Agrupar os dados para média de gasto
media_gasto = df_m3.groupby(['escassez_b', 'Nivel_2', 'Nivel_3', 'Descricao_2',
                             'Descricao_3'])['gasto'].mean().reset_index()

# Função de classificação dos grupos NOVA
def classificar_nova(row):
    desc_3 = str(row['Descricao_3']).lower()
    desc_2 = str(row['Descricao_2']).lower()

    if any(x in desc_3 for x in ['arroz', 'feijão', 'feijao', 'orgânico', 'batata', 'cenoura', 'mandioca',
                                 'cebola', 'tomate', 'alface', 'banana', 'laranja', 'maçã', 'maca',
                                 'frango', 'ovo', 'leite de vaca', 'pescado', 'carne de boi', 'carne de suíno']):
        return 1
    elif 'outras frutas' in desc_3:
        return 1
    elif 'café moído' in desc_3:
        return 1
    elif 'outros' in desc_3 and desc_2 in ['cereais, leguminosas e oleaginosas',
                                           'tubérculos e raízes',
                                           'legumes e verduras',
                                           'frutas',
                                           'carnes, vísceras e pescados',
                                           'aves e ovos']:
        return 1
    elif any(x in desc_3 for x in ['farinha', 'açúcar', 'acucar', 'óleo', 'oleo', 'azeite', 'sal']):
        return 2
    elif 'outras' in desc_3 and desc_2 == 'farinhas, féculas e massas':
        return 2
    elif 'outros' in desc_3 and desc_2 in ['açúcares e derivados', 'óleos e gorduras', 'sal e condimentos']:
        return 2
    elif any(x in desc_3 for x in ['pão', 'queijo']):
        return 3
    elif 'leite em pó' in desc_3:
        return 3
    elif 'outros' in desc_3 and desc_2 in ['panificados', 'leites e derivados']:
        return 3
    elif any(x in desc_3 for x in ['refrigerante', 'biscoito', 'macarrão', 'massa de tomate', 'maionese',
                                   'light', 'diet', 'cerveja', 'bebida', 'industrializados']):
        return 4
    elif 'outras' in desc_3 and desc_2 in ['bebidas e infusões', 'panificados', 'sal e condimentos']:
        return 4
    elif desc_3 == 'outras' and desc_2 == 'bebidas e infusões':
        return 4
    else:
        return None

# Aplicar classificação
media_gasto['grupo_nova'] = media_gasto.apply(classificar_nova, axis=1)

# Agrupar novamente já com o grupo
gasto_produto_nova = media_gasto.groupby(['escassez_b', 'Nivel_2', 'Nivel_3', 'Descricao_2',
                             'Descricao_3', 'grupo_nova'])['gasto'].mean().reset_index()

# Exportar
gasto_produto_nova.to_csv('gasto_por_produto.csv', index=False, encoding='utf-8-sig')


In [ ]:
# Agrupar os dados para média de gasto
media_gasto_quintil = df_m3.groupby(['quintil_renda',
                             'Descricao_3'])['gasto'].mean().reset_index()

# Função de classificação dos grupos NOVA
def classificar_nova(row):
    desc_3 = str(row['Descricao_3']).lower()
    desc_2 = str(row['Descricao_2']).lower()

    if any(x in desc_3 for x in ['arroz', 'feijão', 'feijao', 'orgânico', 'batata', 'cenoura', 'mandioca',
                                 'cebola', 'tomate', 'alface', 'banana', 'laranja', 'maçã', 'maca',
                                 'frango', 'ovo', 'leite de vaca', 'pescado', 'carne de boi', 'carne de suíno']):
        return 1
    elif 'outras frutas' in desc_3:
        return 1
    elif 'café moído' in desc_3:
        return 1
    elif 'outros' in desc_3 and desc_2 in ['cereais, leguminosas e oleaginosas',
                                           'tubérculos e raízes',
                                           'legumes e verduras',
                                           'frutas',
                                           'carnes, vísceras e pescados',
                                           'aves e ovos']:
        return 1
    elif any(x in desc_3 for x in ['farinha', 'açúcar', 'acucar', 'óleo', 'oleo', 'azeite', 'sal']):
        return 2
    elif 'outras' in desc_3 and desc_2 == 'farinhas, féculas e massas':
        return 2
    elif 'outros' in desc_3 and desc_2 in ['açúcares e derivados', 'óleos e gorduras', 'sal e condimentos']:
        return 2
    elif any(x in desc_3 for x in ['pão', 'queijo']):
        return 3
    elif 'leite em pó' in desc_3:
        return 3
    elif 'outros' in desc_3 and desc_2 in ['panificados', 'leites e derivados']:
        return 3
    elif any(x in desc_3 for x in ['refrigerante', 'biscoito', 'macarrão', 'massa de tomate', 'maionese',
                                   'light', 'diet', 'cerveja', 'bebida', 'industrializados']):
        return 4
    elif 'outras' in desc_3 and desc_2 in ['bebidas e infusões', 'panificados', 'sal e condimentos']:
        return 4
    elif desc_3 == 'outras' and desc_2 == 'bebidas e infusões':
        return 4
    else:
        return None

# Aplicar classificação
media_gasto['grupo_nova'] = media_gasto.apply(classificar_nova, axis=1)

# Agrupar novamente já com o grupo
gasto_produto_nova = media_gasto.groupby(['quintil_renda', 'Nivel_2', 'Nivel_3', 'Descricao_2',
                             'Descricao_3', 'grupo_nova'])['gasto'].mean().reset_index()

# Exportar
gasto_produto_nova.to_csv('gasto_por_produto_renda.csv', index=False, encoding='utf-8-sig')


In [ ]:
gasto_produto_nova

,quintil_renda,Nivel_2,Nivel_3,Descricao_2,Descricao_3,grupo_nova,gasto
0,1,101,1011.0,Alimentação light e diet,Alface,1.0,NaN
1,1,101,1011.0,Alimentação light e diet,Alface,2.0,NaN
2,1,101,1011.0,Alimentação light e diet,Alface,3.0,NaN
3,1,101,1011.0,Alimentação light e diet,Alface,4.0,NaN
4,1,101,1011.0,Alimentação light e diet,Arroz,1.0,NaN
...,...,...,...,...,...,...,...
30205435,5,208,1144.0,Óleos e gorduras,Tomate,4.0,NaN
30205436,5,208,1144.0,Óleos e gorduras,Óleo de soja,1.0,NaN
30205437,5,208,1144.0,Óleos e gorduras,Óleo de soja,2.0,NaN
30205438,5,208,1144.0,Óleos e gorduras,Óleo de soja,3.0,NaN


In [ ]:
# 1. Agrupar os dados por quintil e produto
media_gasto = df_m3.groupby(['quintil_renda', 'Descricao_3', 'Nivel_3'])['gasto'].mean().reset_index()

# 2. Função de classificação NOVA
def classificar_nova(desc_3):
    desc_3 = str(desc_3).lower()

    if any(x in desc_3 for x in ['arroz', 'feijão', 'feijao', 'orgânico', 'batata', 'cenoura', 'mandioca',
                                 'cebola', 'tomate', 'alface', 'banana', 'laranja', 'maçã', 'maca',
                                 'frango', 'ovo', 'leite de vaca', 'pescado', 'carne de boi', 'carne de suíno']):
        return 1
    elif 'outras frutas' in desc_3 or 'café moído' in desc_3:
        return 1
    elif any(x in desc_3 for x in ['farinha', 'açúcar', 'acucar', 'óleo', 'oleo', 'azeite', 'sal']):
        return 2
    elif any(x in desc_3 for x in ['pão', 'queijo', 'leite em pó']):
        return 3
    elif any(x in desc_3 for x in ['refrigerante', 'biscoito', 'macarrão', 'massa de tomate', 'maionese',
                                   'light', 'diet', 'cerveja', 'bebida', 'industrializados']):
        return 4
    elif desc_3 == 'outras':
        return 4
    else:
        return None

# 3. Aplicar a classificação NOVA diretamente na coluna Descricao_3
media_gasto['grupo_nova'] = media_gasto['Descricao_3'].apply(classificar_nova)

# 4. Reorganizar as colunas (opcional)
gasto_produto_nova = media_gasto[['quintil_renda', 'Nivel_3', 'Descricao_3', 'grupo_nova', 'gasto']]
# 4.1 Remover linhas com gasto ausente
gasto_produto_nova = gasto_produto_nova.dropna(subset=['gasto'])


# 5. Exportar
gasto_produto_nova.to_csv('gasto_por_produto_renda.csv', index=False, encoding='utf-8-sig')


In [ ]:
df_m3['Descricao_3'].unique()


array([nan, 'Ovo de galinha', 'Açúcar cristal', 'Café moído',
       'Refrigerantes', 'Arroz', 'Farinha de trigo', 'Macarrão', 'Outras',
       'Outras frutas', 'Leite de vaca', 'Outros', 'Biscoito',
       'Pão francês', 'Sal refinado', 'Pescados frescos',
       'Farinha de mandioca', 'Mandioca', 'Leite em pó', 'Frango',
       'Feijão', 'Maçã', 'Cebola', 'Carne de suíno',
       'Carnes e peixes industrializados', 'Óleo de soja', 'Laranja',
       'Tomate', 'Batata inglesa', 'Outros panificados', 'Banana',
       'Carne de boi de primeira', 'Queijos', 'Alface',
       'Carne de boi de segunda', 'Açúcar refinado', 'Cervejas e chopes',
       'Cenoura', 'Maionese', 'Azeite de oliva', 'Light e Diet',
       'Bebidas não alcoólicas light e diet', 'Orgânicos',
       'Massa de tomate', 'Outras bebidas alcoólicas', 'Feijao',
       'Light e diet'], dtype=object)